In [12]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import pandas as pd
import tensorflow as tf
import numpy as np

# Import our input dataset
df_AllYears = pd.read_csv('AllYears_converted.csv')
df_AllYears.head()

,Population,Unemployment,Unsheltered_perc_pop,ES_beds_perc_pop,TS_beds_perc_pop,SH_beds_perc_pop,RRH_units_perc_pop,PSH_units_perc_pop,OPH_units_perc_pop
0,288000.0,4.400000,0.033681,0.277431,0.081250,0.00000,0.059375,0.208681,0.024653
1,425343.0,6.991667,0.041378,0.135185,0.063478,0.00000,0.015752,0.090750,0.000000
2,965787.0,2.600000,0.033755,0.053428,0.028681,0.00352,0.010976,0.180164,0.000000
3,636444.0,3.250000,0.031739,0.041009,0.020897,0.00000,0.004871,0.037710,0.000000
4,209041.0,3.350000,0.096153,0.050708,0.039705,0.00000,0.000000,0.020092,0.003827


In [13]:
q = df_AllYears['Unsheltered_perc_pop'].quantile(np.arange(10) / 10)

In [14]:
df_AllYears['UnshelteredPercentQuantile'] = df_AllYears['Unsheltered_perc_pop'].apply(lambda x : q.index[np.searchsorted(q, x, side='right')-1])

In [15]:
df_AllYears.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2220 entries, 0 to 2219
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Population                  2220 non-null   float64
 1   Unemployment                2220 non-null   float64
 2   Unsheltered_perc_pop        2220 non-null   float64
 3   ES_beds_perc_pop            2220 non-null   float64
 4   TS_beds_perc_pop            2220 non-null   float64
 5   SH_beds_perc_pop            2220 non-null   float64
 6   RRH_units_perc_pop          2220 non-null   float64
 7   PSH_units_perc_pop          2220 non-null   float64
 8   OPH_units_perc_pop          2220 non-null   float64
 9   UnshelteredPercentQuantile  2220 non-null   float64
dtypes: float64(10)
memory usage: 173.6 KB


In [16]:
df_AllYears.head()

,Population,Unemployment,Unsheltered_perc_pop,ES_beds_perc_pop,TS_beds_perc_pop,SH_beds_perc_pop,RRH_units_perc_pop,PSH_units_perc_pop,OPH_units_perc_pop,UnshelteredPercentQuantile
0,288000.0,4.400000,0.033681,0.277431,0.081250,0.00000,0.059375,0.208681,0.024653,0.6
1,425343.0,6.991667,0.041378,0.135185,0.063478,0.00000,0.015752,0.090750,0.000000,0.6
2,965787.0,2.600000,0.033755,0.053428,0.028681,0.00352,0.010976,0.180164,0.000000,0.6
3,636444.0,3.250000,0.031739,0.041009,0.020897,0.00000,0.004871,0.037710,0.000000,0.6
4,209041.0,3.350000,0.096153,0.050708,0.039705,0.00000,0.000000,0.020092,0.003827,0.8


In [17]:
df_AllYears = df_AllYears.drop(['Unsheltered_perc_pop', 'Population', 'ES_beds_perc_pop', 'RRH_units_perc_pop'], axis=1)

In [18]:
df_AllYears.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2220 entries, 0 to 2219
Data columns (total 6 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Unemployment                2220 non-null   float64
 1   TS_beds_perc_pop            2220 non-null   float64
 2   SH_beds_perc_pop            2220 non-null   float64
 3   PSH_units_perc_pop          2220 non-null   float64
 4   OPH_units_perc_pop          2220 non-null   float64
 5   UnshelteredPercentQuantile  2220 non-null   float64
dtypes: float64(6)
memory usage: 104.2 KB


In [19]:
# Remove outcome target from features data
y = df_AllYears.UnshelteredPercentQuantile.values
X = df_AllYears.drop(columns="UnshelteredPercentQuantile").values

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [20]:
# Preprocess numerical data for neural network

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [21]:
# Define the deep learning model 
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=64, activation="sigmoid", input_dim=5))
nn_model.add(tf.keras.layers.Dense(units=64, activation="relu"))
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
53/53 [==============================] - 1s 1ms/step - loss: 0.6807 - accuracy: 0.0949
Epoch 2/100
53/53 [==============================] - 0s 2ms/step - loss: 0.6785 - accuracy: 0.0853
Epoch 3/100
53/53 [==============================] - 0s 1ms/step - loss: 0.6732 - accuracy: 0.0883
Epoch 4/100
53/53 [==============================] - 0s 1ms/step - loss: 0.6720 - accuracy: 0.0919
Epoch 5/100
53/53 [==============================] - 0s 1ms/step - loss: 0.6716 - accuracy: 0.0877
Epoch 6/100
53/53 [==============================] - 0s 1ms/step - loss: 0.6748 - accuracy: 0.0853
Epoch 7/100
53/53 [==============================] - 0s 1ms/step - loss: 0.6705 - accuracy: 0.0853
Epoch 8/100
53/53 [==============================] - 0s 1ms/step - loss: 0.6697 - accuracy: 0.0895
Epoch 9/100
53/53 [==============================] - 0s 1ms/step - loss: 0.6703 - accuracy: 0.0907
Epoch 10/100
53/53 [==============================] - 0s 1ms/step - loss: 0.6725 - accuracy: 0.0871
Epoch 11/

53/53 [==============================] - 0s 2ms/step - loss: 0.6653 - accuracy: 0.0865
Epoch 84/100
53/53 [==============================] - 0s 1ms/step - loss: 0.6630 - accuracy: 0.0901
Epoch 85/100
53/53 [==============================] - 0s 1ms/step - loss: 0.6635 - accuracy: 0.0883
Epoch 86/100
53/53 [==============================] - 0s 2ms/step - loss: 0.6665 - accuracy: 0.0847
Epoch 87/100
53/53 [==============================] - 0s 1ms/step - loss: 0.6635 - accuracy: 0.0901
Epoch 88/100
53/53 [==============================] - 0s 1ms/step - loss: 0.6636 - accuracy: 0.0895
Epoch 89/100
53/53 [==============================] - 0s 1ms/step - loss: 0.6648 - accuracy: 0.0907
Epoch 90/100
53/53 [==============================] - 0s 1ms/step - loss: 0.6646 - accuracy: 0.0919
Epoch 91/100
53/53 [==============================] - 0s 1ms/step - loss: 0.6635 - accuracy: 0.0901
Epoch 92/100
53/53 [==============================] - 0s 1ms/step - loss: 0.6644 - accuracy: 0.0859
Epoch 93/100
